---
layout: global
title: 协同过滤
displayTitle: 协同过滤
---

# 协同过滤

协同过滤[Collaborative filtering](http://en.wikipedia.org/wiki/Recommender_system#Collaborative_filtering)常被用于推荐系统。
这类技术目标在于填充“用户－商品”联系矩阵中的缺失项。
`spark.ml`目前支持基于模型的协同过滤，其中用户和商品以少量的隐因子来描述，用以预测缺失项。
`spark.ml`使用交替最小二乘[alternating least squares(ALS)](http://dl.acm.org/citation.cfm?id=1608614)算法来学习这些隐因子。
`spark.ml`实现的接口有这些参数：

* *numBlocks*是用户和商品将被划分为几块以并行计算（默认10）。
* *rank*是模型中的隐因子数量（默认10）。
* *maxIter*最大迭代轮数（默认10）。
* *regParam*指定ALS中的正则化参数（默认1.0）。
* *implicitPrefs*指定使用*显式反馈*还是*隐式反馈*数据的ALS变体（默认为`false`表示使用*显式反馈*）。
* *alpha*用于隐式反馈的ALS，影响观测到的偏好中的*baseline*置信度（默认1.0）。
* *nonnegative*指定是否对最小二乘应用非负约束（默认为`false`）。

可以调整这些参数，不断优化结果，使均方差变小。比如：imaxIter越大，regParam越小，均方差会越小，推荐结果较优。

**注意**：基于DataFrame的ALS接口目前仅支持整数型的用户和商品编号。
也支持用户和商品编号列的其他数值类型，但编号必须在整数型范围内。

## 什么是ALS

ALS通过观察到的所有用户给商品的打分，来推断每个用户的喜好并向用户推荐适合的商品。
举个例子，我们看下面一个`8*8`的用户打分矩阵：

<div  align="center"><img src="img/ALS.1.1.png" width = "450" height = "300" alt="8*8打分" align="center" /></div>

这个矩阵$A$的每一行代表一个用户$(u_1,u_2,...,u_8)$、每一列代表一个商品$(v_1,v_2,...,v_8)$、用户的打分为`1-9`分。
这个矩阵只显示了观察到的打分，我们需要推测没有观察到的打分。比如$(u_6,v_5)$打分多少？
如果以数独的方式来解决这个问题，可以得到唯一的结果。
因为数独的规则很强，每添加一条规则，就让整个系统的自由度下降一个量级。
当我们满足所有的规则时，整个系统的自由度就降为$1$了，也就得出了唯一的结果。

对于上面的打分矩阵，如果我们不添加任何条件的话，也即打分之间是相互独立的，我们就没法得到$(u_6,v_5)$的打分。
所以在这个用户打分矩阵的基础上，我们需要提出一个限制其自由度的合理假设，使得我们可以通过观察已有打分来猜测未知打分。

ALS的核心就是这样一个假设：打分矩阵是近似低秩的。
换句话说，就是一个$A_{m*n}$的打分矩阵可以由分解的两个小矩阵$U_{m*k}$和$V_{k*n}$的乘积来近似，即$A=U{V}^{T},k <= m,n$。
这就是ALS的矩阵分解方法。这样我们把系统的自由度从$O(mn)$降到了$O((m+n)k)$。

那么ALS的低秩假设为什么是合理的呢？
我们描述一个人的喜好经常是在一个抽象的低维空间上进行的，并不需要一一列出他喜好的事物。
例如，我喜好看侦探影片，可能代表我喜欢《神探夏洛特》、《神探狄仁杰》等。
这些影片都符合我对自己喜好的描述，也就是说他们在这个抽象的低维空间的投影和我的喜好相似。

再抽象一些来描述这个问题，我们把某个人的喜好映射到了低维向量$u_i$上，同时将某个影片的特征映射到了维度相同的向量$v_j$上，那么这个人和这个影片的相似度就可以表述成这两个向量之间的内积$u_{i}^{T}v_{j}$ 。
我们把打分理解成相似度，那么打分矩阵$A$就可以由用户喜好矩阵和产品特征矩阵的乘积$U{V}^{T}$来近似了。

低维空间的选取是一个问题。
这个低维空间要能够很好的区分事物，那么就需要一个明确的可量化目标，这就是重构误差。
在ALS中我们使用F范数来量化重构误差，就是每个元素重构误差的平方和。
这里存在一个问题，我们只观察到部分打分，$A$中的大量未知元是我们想推断的，所以这个重构误差是包含未知数的。
解决方案很简单：只计算已知打分的重构误差。

$$\sum\limits_{(i,j)\in R}(a_{ij} - u_iv_j^T)^2$$

下一节我们将从原理上讲解`spark.ml`中实现的ALS模型。

## Spark中ALS的实现原理

`spark.ml`利用交换最小二乘解决矩阵分解问题，分两种情况：数据集是显式反馈和数据集是隐式反馈。
由于隐式反馈算法的原理是在显示反馈算法原理的基础上作的修改，所以我们在此只会具体讲解数据集为隐式反馈的算法。

### 介绍

从广义上讲，推荐系统基于两种不同的策略：基于内容的方法和基于协同过滤的方法。
`spark.ml`中使用协同过滤的方式。
协同过滤分析用户以及用户相关的产品的相关性，用以识别新的用户-产品相关性。
协同过滤系统需要的唯一信息是用户过去的行为信息，比如对产品的评价信息。
协同过滤是领域无关的，所以它可以方便解决基于内容方法难以解决的许多问题。

推荐系统依赖不同类型的输入数据，最方便的是高质量的显式反馈数据，它们包含用户对感兴趣商品明确的评价。
例如，Netflix收集的用户对电影评价的星星等级数据。
但是显式反馈数据不一定总是找得到，因此推荐系统可以从更丰富的隐式反馈信息中推测用户的偏好。
隐式反馈类型包括购买历史、浏览历史、搜索模式甚至鼠标动作。
例如，购买同一个作者许多书的用户可能喜欢这个作者。

许多研究都集中在处理显式反馈，然而在很多应用场景下，应用程序重点关注隐式反馈数据。因为可能用户不愿意评价商品或者由于系统限制我们不能收集显式反馈数据。在隐式模型中，一旦用户允许收集可用的数据，在客户端并不需要额外的显式数据。文献中的系统避免主动地向用户收集显式反馈信息，所以系统仅仅依靠隐式信息。

了解隐式反馈的特点非常重要，因为这些特质使我们避免了直接调用基于显式反馈的算法。最主要的特点有如下几种：

1. 没有负反馈。通过观察用户行为，我们可以推测那个商品他可能喜欢，然后购买，但是我们很难推测哪个商品用户不喜欢。这在显式反馈算法中并不存在，因为用户明确告诉了我们哪些他喜欢哪些他不喜欢。

1. 隐式反馈是内在的噪音。虽然我们拼命的追踪用户行为，但是我们仅仅只是猜测他们的偏好和真实动机。例如，我们可能知道一个人的购买行为，但是这并不能完全说明偏好和动机，因为这个商品可能作为礼物被购买而用户并不喜欢它。

1. 显示反馈的数值值表示偏好（preference），隐式回馈的数值值表示信度（confidence）。基于显示反馈的系统用星星等级让用户表达他们的喜好程度，例如一颗星表示很不喜欢，五颗星表示非常喜欢。基于隐式反馈的数值值描述的是动作的频率，例如用户购买特定商品的次数。一个较大的值并不能表明更多的偏爱。但是这个值是有用的，它描述了在一个特定观察中的信任度。一个发生一次的事件可能对用户偏爱没有用，但是一个周期性事件更可能反映一个用户的选择。

1. 评价隐式反馈推荐系统需要合适的手段。

### 显式反馈模型

隐因子模型由一个针对协同过滤的交替方法组成，它以一个更加全面的方式发现潜在特征来解释观察的`ratings`数据。
我们关注的模型由奇异值分解（SVD）推演而来。
一个典型的模型将每个用户$u$（包含一个用户-特征向量$u_i$）和每个商品$v$（包含一个商品-特征向量$v_j$）联系起来。
预测通过内积$r_{ij}=u_{i}^{T}v_{j}$来实现。
另一个需要关注的地方是参数估计。
许多当前的工作都应用到了显式反馈数据集中，这些模型仅仅基于观察到的`rating`数据直接建模，同时通过一个适当的正则化来避免过拟合。
公式如下：

$$\min_{u,v}\sum\limits_{(i,j)\in R}[(a_{ij} - u_iv_j^T)^2+\lambda(u_i^2+v_j^2)]$$

在公式(2.1)中，$\lambda$是正则化的参数，正规化是为了防止过拟合的情况发生。
这样，我们用最小化重构误差来解决协同推荐问题。我们也成功将推荐问题转换为了最优化问题。


### 隐式反馈模型

在显式反馈的基础上，我们需要做一些改动得到我们的隐式反馈模型。
`spark.ml`使用[Collaborative Filtering for Implicit Feedback Datasets](http://dx.doi.org/10.1109/ICDM.2008.22)提出的方法来处理隐式反馈数据。
首先，我们需要形式化由$r_{ij}$变量衡量的信任度的概念。
我们引入了一组二元变量$p_{ij}$ ，它表示用户$u$对商品$v$的偏好。
$p_{ij}$的公式如下：

$$\large p_{ij}= \Bigg\{ ^{1,  r_{ij}>0}_{0,r_{ij}=0}$$

换句话说，如果用户购买了商品，我们认为用户喜欢该商品，否则我们认为用户不喜欢该商品。
然而我们的信念（`beliefs`）与变化的信任（`confidence`）等级息息相关。
首先，很自然的，$p_{ij}$的值为0和低信任有关。
用户对一个商品没有得到一个正的偏好可能源于多方面的原因，并不一定是不喜欢该商品。
例如，用户可能并不知道该商品的存在。
另外，用户购买一个商品也并不一定是用户喜欢它。
因此我们需要一个新的信任等级来显示用户偏爱某个商品。
一般情况下，$r_{ij}$越大，越能暗示用户喜欢某个商品。
因此，我们引入了一组变量$c_{ij}$，它衡量了我们观察到$p_{ij}$的信任度。
$c_{ij}$一个合理的选择如下所示：

$$\large c_{ij} = 1 + \alpha r_{ij}$$

按照这种方式，我们存在最小限度的信任度，并且随着我们观察到的正偏向的证据越来越多，信任度也会越来越大。

我们的目的是找到用户向量$u_i$以及商品向量$v_j$来表明用户偏好。这些向量分别是用户因素（特征）向量和商品因素（特征）向量。
本质上，这些向量将用户和商品映射到一个公用的隐式因素空间，从而使它们可以直接比较。
这和用于显式数据集的矩阵分解技术类似，但是包含两点不一样的地方：

1. 我们需要考虑不同的信任度，
1. 最优化需要考虑所有可能的$u, v$对，而不仅仅是和观察数据相关的$u, v$对。

显性反馈的矩阵分解优化时，对于`missing data`(没有评分)，是不会当做训练数据输入到模型的，优化时针对已知评分数据优化。
而这里隐性反馈，是利用所有可能的$u, v$键值对，所以总的数据是$m*n$，其中$m$是用户数量，$n$是物品数量。
这里没有所谓的`missing data`，因为假如$u$对$v$没有任何动作，我们就认为偏好值为0，只不过置信度较低而已。
因此，通过最小化下面的损失函数来计算相关因子。

$$min_{u,v}\sum _{i,j}c_{ij}(p_{ij}-u_{i}^{T}v_{j})^{2} + \lambda (\sum_{i}\left \| u_{i} \right \|^{2} + \sum_{j}\left \|v_{j} \right \|^{2})$$

### 求解最小化损失函数

考虑到损失函数包含$m*n$个元素，$m$是用户的数量，$n$是商品的数量。
一般情况下，$m*n$可以到达几百亿。
这么多的元素应该避免使用随机梯度下降法来求解，因此，spark选择使用交替最优化方式求解。

上述损失函数是非凸函数，无法求解最优解。
但是，固定公式中的用户-特征向量或者商品-特征向量，公式就会变成二次方程，可以求出全局的极小值。
交替最小二乘的计算过程是：交替的重新计算用户-特征向量和商品-特征向量，每一步都保证降低损失函数的值，直到找到极小值。
交替最小二乘法的处理过程如下所示：

* 先随机生成一个$U_{m*k}^{(0)}$。一般可以取0值或者全局均值。
* 固定$U^{(0)}$，即认为是已知的常量，来求解：
$$\large C = \sum\limits_{(i,j)\in R}[(a_{ij} - u_i^{(0)}v_j^T)^2+\lambda((u_i^2)^{(0)}+v_j^2)]$$
由于上式中只有$v_j$一个未知变量，因此$C$的最优化问题转化为最小二乘问题，用最小二乘法求解$v_j$的最优解： 
固定$j,j\in(1,2,...,n)$，等式两边关于$v_j$求导得：
$$
\begin{align}
&\large \frac{d(c)}{d(v_j)}\\
&\large= \frac{d}{d(v_j)}(\sum\limits_{i=1}^{m}[(a_{ij} - u_i^{(0)}v_j^T)^2+\lambda((u_i^2)^{(0)}+v_j^2)])\\
&\large= \sum\limits_{i=1}^m[2(a_{ij} - u_i^{(0)}v_j^T)(- (u_i^T)^{(0)})+2\lambda v_j]\\
&\large= 2\sum\limits_{i=1}^m[( u_i^{(0)}(u_i^T)^{(0)}+\lambda)v_j-a_{ij}(u_i^T)^{(0)}]
\end{align}
$$
令$\large \frac{d(c)}{d(v_j)} =0$，可得：
$$
\begin{align}
&\large\sum\limits_{i=1}^m[( u_i^{(0)}(u_i^T)^{(0)}+\lambda)v_j]\\
&\large=\sum\limits_{i=1}^m a_{ij}(u_i^T)^{(0)}\\
&\large=> (U^{(0)}(U^T)^{(0)} + \lambda E)v_j\large= a_j^TU^{(0)}
\end{align}
$$
令$M_1 = U^{(0)}(U^T)^{(0)} + \lambda E , M_2 = a_j^TU^{(0)}$，则$v_j = M_1^{-1}M_2$

* 按照上式依次计算$v_1，v_2，...，v_n$，从而得到$V^{(0)}$
* 同理，用步骤2中类似的方法: 
$$\large C = \sum\limits_{(i,j)\in R}[(a_{ij} - u_i(v_j^T)^{(0)})^2+\lambda(u_i^2+(v_j^2)^{(0)})]$$
固定$i , i\in (1,2,...,m)$，等式两边关于$u_i$求导得：
$$
\begin{align}
&\large \frac{d(c)}{d(u_i)}\\
&\large= \frac{d}{d(u_i)}(\sum\limits_{j=1}^{n}[(a_{ij} - u_i(v_j^T)^{(0)})^2+\lambda((u_i^2)+(v_j^2)^{(0)})])\\
&\large= \sum\limits_{j=1}^n[2(a_{ij} - u_i(v_j^T)^{(0)})(- (v_j^T)^{(0)})+2\lambda u_i]\\
&\large= 2\sum\limits_{j=1}^n[( v_j^{(0)}(v_j^T)^{(0)}+\lambda)u_i-a_{ij}(v_j^T)^{(0)}]
\end{align}
$$
令$\large \frac{d(c)}{d(u_i)} =0$，可得：
$$
\begin{align}
&\large\sum\limits_{j=1}^n[( v_j^{(0)}(v_j^T)^{(0)}+\lambda)u_i]\\
&\large=\sum\limits_{j=1}^n a_{ij}(v_j^T)^{(0)}\\
&\large=>( (V^{(0)}(V^T)^{(0)} + \lambda E)u_i = a_i^TV^{(0)}
\end{align}
$$
令$M_1 = V^{(0)}(V^T)^{(0)} + \lambda E , M_2 =a_i^TV^{(0)}$，则$u_i = M_1^{-1}M_2$

* 按照上式依次计算$u_1，u_2，...，u_n$，从而得到$U^{(1)}$
* 循环执行步骤2、3，直到损失函数$C$的值收敛（或者设置一个迭代次数N，迭代执行步骤2、3，N次后停止）。这样，就得到了$C$最优解对应的矩阵$U$、$V$。


## 正则化参数

我们调整正则化参数`regParam`来解决用户在更新用户因子时产生新评分或者商品更新商品因子时收到的新评分带来的最小二乘问题。
这个方法叫做“ALS-WR”来自于“[Large-Scale Parallel Collaborative Filtering for the Netflix Prize](http://dx.doi.org/10.1007/978-3-540-68880-8_32)”。
它降低`regParam`对数据集规模的依赖，所以我们可以将从部分子集中学习到的最佳参数应用到整个数据集中时获得同样的性能。

## 冷启动策略

当使用`ALSModel`预测时会经常遇到测试数据集中的用户和/或商品没有在训练时出现。这种情况通常出现在两种场景里：

1. 生产系统中，新用户和商品没有打分历史所以模型没有为此训练过（即“冷启动问题”）。
2. 在交叉验证时，数据分割成训练和评估两部分。若使用`CrossValidator`或`TrainValidationSplit`之类的简单随机分割，经常遇到评估数据集中的用户和/或商品没有在训练数据中出现。

当用户和/或商品因子没有被模型表示时，Spark`ALSModel.transform`默认预测为`NaN`。生产系统中这可能有用，因为这指示了新用户和商品，这样系统可以提出些备选项作为预测。

当然交叉验证时这样不合适，任何`NaN`预测值都会导致评估指标变成`NaN`（比如使用`RegressionEvaluator`）。
这样就不能做模型选择了。

Spark允许用户设置`coldStartStrategy`参数来抛弃`DataFrame`中任何包含`NaN`预测值的行。
评估指标就只考虑非`NaN`数据所以不会失效。
下面的例子展示了如何使用该参数。

**注意**：目前支持的冷启动策略有`"nan"`（上面描述的默认行为）和`"drop"`。将来可能会支持更多策略。

**样例**

下面的例子中我们从[MovieLens dataset](http://grouplens.org/datasets/movielens/)载入评分数据，每一行包括一个用户、一个电影、一个评分和一个时间戳。
我们默认其评分是显式的来训练ALS模型（`implicitPrefs`设为`False`）。
我们通过预测评分的均方根误差来评价推荐模型。

请参考[`ALS` Python文档](api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS)了解更多细节。

完整样例代码可以在[Spark仓库](https://github.com/apache/spark)中的"examples/src/main/python/ml/als_example.py"找到

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text("data/mllib/als/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=long(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

如果评分矩阵来自其他信息来源，也可将`implicitPrefs`设置为`true`来获得更好的结果。

In [ ]:
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True,
          userCol="userId", itemCol="movieId", ratingCol="rating")